# 2. Data pre processing

In [1]:
import pandas as pd
import numpy as np
import re
import time
import json

import requests as rq
import bs4 as bs4
import tqdm 
import glob
import feather

In [3]:
df = pd.read_json("parsed_videos.json", lines=True)

In [4]:
list_of_links = df['link'].unique()

In [5]:
url = 'https://www.youtube.com{link}'

for link in tqdm.tqdm_notebook(list_of_links):
    urll = url.format(link=link)
    response = rq.get(urll)
    
    link_name = re.search("v=(.*)", link).group(1)
    
    with open("./raw_data/videos/video_{}.html".format(link_name), 'w+') as output:
        output.write(response.text)

/home/lucifer/DataScience/boss/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
with open("parsed_videos_info.json", 'w+') as output:
    for video in tqdm.tqdm_notebook(sorted(glob.glob("./raw_data/videos/*"))):
        with open(video, 'r+') as inp:
            page_html = inp.read()
            parsed = bs4.BeautifulSoup(page_html, 'html.parser')
            
            class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
            id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
            channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
            meta = parsed.find_all("meta")
            
            data = dict()
            
            for e in class_watch:
                colname = "_".join(e['class'])
                if "clearfix" in colname:
                    continue
                data[colname] = e.text.strip()
            
            for e in id_watch:
                colname = e['id']
                data[colname] = e.text.strip()
            
            for e in meta:
                colname = e.get('property')
                if colname is not None:
                    data[colname] = e['content']
                    
            for link_num, e in enumerate(channel):
                data["channel_link_{}".format(link_num)] = e['href']
                
            output.write("{}\n".format(json.dumps(data)))

/home/lucifer/DataScience/boss/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Verificação

In [3]:
df = pd.read_json("parsed_videos_info.json", lines=True)
df.shape

(1615, 174)

In [6]:
useful_col = ['watch-title', 'og:url', 'watch-view-count', 'watch-time-text', 'content_watch-info-tag-list', 'watch7-headline', 'watch7-user-header', 'watch8-sentiment-actions', 'og:image', 'og:image:width', 'og:image:height', 'og:description', 'og:video:width', 'og:video:height', 'og:video:tag', 'channel_link_0']

In [7]:
df[useful_col].head(2)

,watch-title,og:url,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:tag,channel_link_0
0,#DataScience #Pandas #python Python Pandas Tut...,https://www.youtube.com/watch?v=--EdOZqByHo,62 visualizações,Publicado em 11 de abr. de 2020,Educação,#DataScience #Pandas #python Python Pandas Tut...,Code Mania\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarregan...,62 visualizações\n\n\n\n\n\n\n\n4\n\nGostou de...,https://i.ytimg.com/vi/--EdOZqByHo/hqdefault.jpg,480,360,This pandas tutorial covers basics on datafram...,640.0,360.0,python data science tutorial,/channel/UCiO8B22LQBecxz9JjYrk7yA
1,Machine Learning Course A To Z || Beginner to ...,https://www.youtube.com/watch?v=-58kO_zYUGE,174.642 visualizações,Publicado em 10 de ago. de 2018,Educação,Machine Learning Course A To Z || Beginner to ...,Geek's Lesson\n\n\n\n\n\n\n\n\n\n\n\n\n\nCarre...,174.642 visualizações\n\n\n\n\n\n\n\n5.121\n\n...,https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...,1280,720,Welcome to this free online class on machine l...,640.0,360.0,Ai and machine learning course,/channel/UCKXx22vOENUyHrVAADq7Z_g


In [25]:
df[useful_col].to_feather("raw_data.feather")

In [26]:
df[useful_col].to_csv("raw_data_without_labels.csv")